# Imports necessários

## Bibliotecas

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt 

import statistics as st
import sys
import math
import pandas as pd
import numpy as np
sys.path.insert(1, '..\Modelo')


## Importando a série

In [26]:
df = pd.read_csv("../Dados/Dados refinados/timeseries.csv", index_col= 0)
df.head(10)

,mes,hora_sin,hora_cos,dia_semana_sin,dia_semana_cos,QTD_VEICULOS_USINA,primeira_quinzena,feriado,fim_semana,lag_1h,lag_1d,lag_7d,TPV
0,1,9.659258e-01,-0.258819,-0.974928,-0.222521,21,1,0,1,8.840000,6.538125,2.220000,8.484063
1,1,8.660254e-01,-0.500000,-0.974928,-0.222521,23,1,0,1,8.885333,6.293235,2.220000,8.150294
2,1,8.660254e-01,-0.500000,-0.974928,-0.222521,28,1,0,1,8.484063,6.093056,2.476667,7.830648
3,1,7.071068e-01,-0.707107,-0.974928,-0.222521,35,1,0,1,8.150294,5.945197,2.605000,7.581667
4,1,7.071068e-01,-0.707107,-0.974928,-0.222521,32,1,0,1,7.830648,5.780604,2.682000,7.362583
5,1,5.000000e-01,-0.866025,-0.974928,-0.222521,29,1,0,1,7.581667,5.620099,2.733333,7.186905
6,1,5.000000e-01,-0.866025,-0.974928,-0.222521,29,1,0,1,7.362583,5.523352,2.770000,6.978712
7,1,2.588190e-01,-0.965926,-0.974928,-0.222521,30,1,0,1,7.186905,5.412337,2.797500,6.898043
8,1,2.588190e-01,-0.965926,-0.974928,-0.222521,32,1,0,1,6.978712,5.309656,2.818889,6.772153
9,1,1.224647e-16,-1.000000,-0.974928,-0.222521,31,1,0,1,6.898043,5.209537,2.808000,6.632307


# Transformação de dados

## Normalizando os dados

In [27]:
#Separando X e y

X = df.iloc[:, :-1]
y = df.iloc[:,-1]

In [28]:
#Definindo o scalers
scaler_X, scaler_y = MinMaxScaler(),MinMaxScaler()

#fit
scaler_X.fit(X[['mes','QTD_VEICULOS_USINA']])
scaler_y.fit(pd.DataFrame(y))

#transform

X[['mes','QTD_VEICULOS_USINA']] = scaler_X.transform(X[['mes','QTD_VEICULOS_USINA']])
y = scaler_y.transform(pd.DataFrame(y))

In [29]:
cat_cols = ['mes', 'primeira_quinzena', 'feriado', 'fim_semana']

for col in cat_cols:
    X[col] = X[col].astype('category')

# Cross-validation (Timeseries)

In [30]:
ts_cv = TimeSeriesSplit(n_splits=5)
all_splits = list(ts_cv.split(X, y))

In [31]:
all_splits

[(array([   0,    1,    2, ..., 1713, 1714, 1715]),
  array([1716, 1717, 1718, ..., 3424, 3425, 3426])),
 (array([   0,    1,    2, ..., 3424, 3425, 3426]),
  array([3427, 3428, 3429, ..., 5135, 5136, 5137])),
 (array([   0,    1,    2, ..., 5135, 5136, 5137]),
  array([5138, 5139, 5140, ..., 6846, 6847, 6848])),
 (array([   0,    1,    2, ..., 6846, 6847, 6848]),
  array([6849, 6850, 6851, ..., 8557, 8558, 8559])),
 (array([   0,    1,    2, ..., 8557, 8558, 8559]),
  array([ 8560,  8561,  8562, ..., 10268, 10269, 10270]))]

# Modelagem


## Gradient Boosting Model

In [32]:
from sklearn.ensemble import HistGradientBoostingRegressor

gbrt = HistGradientBoostingRegressor(categorical_features="from_dtype", random_state=42)
categorical_columns = X.columns[X.dtypes == "category"]
print("Categorical features:", categorical_columns.tolist())

Categorical features: ['mes', 'primeira_quinzena', 'feriado', 'fim_semana']


In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def avaliaModelo_GBR(X, y, splits = all_splits):

    print("Cross-validation - GBR")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        model = HistGradientBoostingRegressor(categorical_features="from_dtype", random_state=42)
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(predictions.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )


In [34]:
avaliaModelo_GBR(X.values, y.ravel(), splits = all_splits)

Cross-validation - GBR
---------------------------------------------------------------------------------

[1/5] X_train: (1716, 12), y_train: (1716,), X_test: (1711, 12), y_test: (1711,)
MAE: 0.32295892358272715 | RMSE: 0.6324196413469126


[2/5] X_train: (3427, 12), y_train: (3427,), X_test: (1711, 12), y_test: (1711,)
MAE: 0.2676292196786708 | RMSE: 0.5206069151480549


[3/5] X_train: (5138, 12), y_train: (5138,), X_test: (1711, 12), y_test: (1711,)
MAE: 0.2903534699764067 | RMSE: 0.6511521194630381


[4/5] X_train: (6849, 12), y_train: (6849,), X_test: (1711, 12), y_test: (1711,)
MAE: 0.650587187615871 | RMSE: 2.9578345469536815


[5/5] X_train: (8560, 12), y_train: (8560,), X_test: (1711, 12), y_test: (1711,)
MAE: 0.18054885692255598 | RMSE: 0.42197415318561377


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 0.342 +/- 0.161
RMSE: 1.037 +/- 0.964


## eTS

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def avaliaModelo_eTS(X, y, splits = all_splits):

    print("Cross-validation - eTS")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from eTS import eTS
        model = eTS()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )


In [36]:
avaliaModelo_eTS(X.values, y, splits = all_splits)

Cross-validation - eTS
---------------------------------------------------------------------------------

[1/5] X_train: (1716, 12), y_train: (1716, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.3801168734667584 | RMSE: 0.6124043612948069


[2/5] X_train: (3427, 12), y_train: (3427, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.33684965883939033 | RMSE: 0.670318152336903


[3/5] X_train: (5138, 12), y_train: (5138, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.39617900532783495 | RMSE: 0.7780786305559725


[4/5] X_train: (6849, 12), y_train: (6849, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.43880689257243727 | RMSE: 1.0665782756677618


[5/5] X_train: (8560, 12), y_train: (8560, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.34332335415384 | RMSE: 0.6513461899198593


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 0.379 +/- 0.037
RMSE: 0.756 +/- 0.165


## Simpl_eTS

In [37]:
def avaliaModelo_Simpl_eTS(X, y, splits = all_splits):

    print("Cross-validation - Simpl_eTS")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from Simpl_eTS import Simpl_eTS
        model = Simpl_eTS()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )

In [38]:
avaliaModelo_Simpl_eTS(X.values, y, splits = all_splits)

Cross-validation - Simpl_eTS
---------------------------------------------------------------------------------

[1/5] X_train: (1716, 12), y_train: (1716, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.6825639631240574 | RMSE: 1.0562772546071006


[2/5] X_train: (3427, 12), y_train: (3427, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.36509122992683074 | RMSE: 0.6440663168799289


[3/5] X_train: (5138, 12), y_train: (5138, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.3781812702991797 | RMSE: 0.7229477705805197


[4/5] X_train: (6849, 12), y_train: (6849, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.5803687326685356 | RMSE: 1.3912690180563967


[5/5] X_train: (8560, 12), y_train: (8560, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.5303923228217456 | RMSE: 0.91570562939726


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 0.507 +/- 0.121
RMSE: 0.946 +/- 0.265


## exTS

In [39]:
def avaliaModelo_exTS(X, y, splits = all_splits):

    print("Cross-validation - exTS")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from exTS import exTS
        model = exTS()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )

In [40]:
avaliaModelo_exTS(X.values, y, splits = all_splits)

Cross-validation - exTS
---------------------------------------------------------------------------------

[1/5] X_train: (1716, 12), y_train: (1716, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 1.960944784241269 | RMSE: 2.5097038145972115


[2/5] X_train: (3427, 12), y_train: (3427, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.3689643226400376 | RMSE: 0.6858175232122266


[3/5] X_train: (5138, 12), y_train: (5138, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.5165108561945241 | RMSE: 1.070440594326642


[4/5] X_train: (6849, 12), y_train: (6849, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.5785143468105615 | RMSE: 1.4086274796518043


[5/5] X_train: (8560, 12), y_train: (8560, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.6440537484571429 | RMSE: 1.6232801730028308


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 0.814 +/- 0.581
RMSE: 1.460 +/- 0.614


## eMG

In [41]:
def avaliaModelo_eMG(X, y, splits = all_splits):

    print("Cross-validation - eMG")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from eMG import eMG
        model = eMG()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )

In [42]:
avaliaModelo_eMG(X.values, y, splits = all_splits)

Cross-validation - eMG
---------------------------------------------------------------------------------

[1/5] X_train: (1716, 12), y_train: (1716, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.6748641086075853 | RMSE: 1.0018437844224195


[2/5] X_train: (3427, 12), y_train: (3427, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.8298112518279915 | RMSE: 1.3111717332802855


[3/5] X_train: (5138, 12), y_train: (5138, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 0.8008209173389094 | RMSE: 1.338752392108105


[4/5] X_train: (6849, 12), y_train: (6849, 1), X_test: (1711, 12), y_test: (1711, 1)
MAE: 1.4924836987083119 | RMSE: 4.753329764153785


[5/5] X_train: (8560, 12), y_train: (8560, 1), X_test: (1711, 12), y_test: (1711, 1)


## ePL+

In [ ]:
def avaliaModelo_ePLplus(X, y, splits = all_splits):

    print("Cross-validation - ePL+")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from ePL_plus import ePL_plus
        model = ePL_plus()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )

In [ ]:
avaliaModelo_ePLplus(X.values, y, splits = all_splits)

Cross-validation - ePL+
---------------------------------------------------------------------------------

[1/5] X_train: (2175, 9), y_train: (2175, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 4.476199927685411 | RMSE: 5.960284365400384


[2/5] X_train: (4348, 9), y_train: (4348, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 2.926509703011955 | RMSE: 3.8262618343422528


[3/5] X_train: (6521, 9), y_train: (6521, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 6.446553854642832 | RMSE: 8.514242599109009


[4/5] X_train: (8694, 9), y_train: (8694, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 3.1543957381500194 | RMSE: 4.061888619096644


[5/5] X_train: (10867, 9), y_train: (10867, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 1.5013914935197874 | RMSE: 1.9759981189786


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 3.701 +/- 1.666
RMSE: 4.868 +/- 2.218


## ePL_KRLS_DISCO


In [ ]:
lower_rmse = np.inf
for param in grid:
    
    #print(param)

    # Optimize parameters
    model = ePL_KRLS_DISCO(**param)
    model.fit(X_train,y_train)
    # Make predictions
    y_pred = model.predict(X_val)
    
    
    # Calculating the error metrics
    # Compute the Root Mean Square Error
    RMSE = math.sqrt(mean_squared_error(y_val, y_pred))
    
    if RMSE < lower_rmse:
        lower_rmse = RMSE
        best_ePL_KRLS_DISCO_params = param

# Optimized parameters
model = ePL_KRLS_DISCO(**best_ePL_KRLS_DISCO_params)
OutputTraining, Rules = model.fit(X_train,y_train)
# Make predictions
y_pred = model.predict(X_test)

# Save predictions to dataframe
predictions[f'{Model_Name}'] = y_pred

# Model name
print(f'\nResults for {Model_Name}: \n')

# Calculating the error metrics
# Compute the Root Mean Square Error
RMSE = math.sqrt(mean_squared_error(y_test, y_pred))
print("\nRMSE:", RMSE)
# Compute the Normalized Root Mean Square Error
NRMSE = RegressionMetric(y_test, y_pred).normalized_root_mean_square_error()
print("NRMSE:", NRMSE)
# Compute the Non-Dimensional Error Index
NDEI= RMSE/st.stdev(np.asfarray(y_test.flatten()))
print("NDEI:", NDEI)
# Compute the Mean Absolute Error
MAE = mean_absolute_error(y_test, y_pred)
print("MAE:", MAE)
# Compute the Mean Absolute Percentage Error
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print("MAPE:", MAPE)
# Number of rules
print("Rules:", Rules[-1])
   
ePL_KRLS_DISCO_ = f'{Model_Name} & {NRMSE:.2f} & {NDEI:.2f} & {MAPE:.2f} & {Rules[-1]}'

#print(f'\n{best_ePL_KRLS_DISCO_params}')

# Store results to dataframe
newrow = pd.DataFrame([[Model_Name, NRMSE, NDEI, MAPE, Rules[-1], best_ePL_KRLS_DISCO_params]], columns=columns)
results = pd.concat([results, newrow], ignore_index=True)

In [ ]:
def avaliaModelo_ePL_KRLS_DISCO(X, y, splits = all_splits):

    print("Cross-validation - ePL_KRLS_DISCO")
    print("---------------------------------------------------------------------------------\n")
    mae_scores = np.array([])
    rmse_scores = np.array([])

    for i, (train_index, test_index) in enumerate(all_splits):

        #Chamando a função a cada iteração pq a função predict dá append no array sem zerar antes
        from ePL_KRLS_DISCO import ePL_KRLS_DISCO
        model = ePL_KRLS_DISCO()
        
        tmp_y_test = np.array([])
        tmp_prev = np.array([])
        OutputTraining = np.array([])
        Rules = np.array([])
        OutputTest = np.array([])

        #Separando os dados (já normalizados)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print(f"[{i+1}/5] X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")

        OutputTraining, Rules = model.fit(X_train, y_train)
        OutputTest = model.predict(X_test)
        

        #desnormalizando os dados temporariamente

        tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

        #Calculando as métricas de MAE e RMSE

        mae = mean_absolute_error(tmp_y_test, tmp_prev)
        rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

        print(f"MAE: {mae} | RMSE: {rmse}")
        print("\n")

        mae_scores = np.append(mae_scores, mae)
        rmse_scores = np.append(rmse_scores, rmse)

    print("RESULTADO FINAL DA AVALIAÇÃO")
    print("---------------------------------------------------------------------------------")
    print(
    f"MAE: {mae_scores.mean():.3f} +/- {mae_scores.std():.3f}\n"
    f"RMSE: {rmse_scores.mean():.3f} +/- {rmse_scores.std():.3f}"
    )

In [ ]:
avaliaModelo_ePL_KRLS_DISCO(X.values, y, splits = all_splits)

Cross-validation - ePL_KRLS_DISCO
---------------------------------------------------------------------------------

[1/5] X_train: (2175, 9), y_train: (2175, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 2.2255082008593807 | RMSE: 3.5015844969237717


[2/5] X_train: (4348, 9), y_train: (4348, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 4.764533981365316 | RMSE: 7.073927122621309


[3/5] X_train: (6521, 9), y_train: (6521, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 7.677546672441963 | RMSE: 12.398542740382728


[4/5] X_train: (8694, 9), y_train: (8694, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 11.475212316001109 | RMSE: 18.698495182771126


[5/5] X_train: (10867, 9), y_train: (10867, 1), X_test: (2173, 9), y_test: (2173, 1)
MAE: 14.172348050634499 | RMSE: 23.62254371733754


RESULTADO FINAL DA AVALIAÇÃO
---------------------------------------------------------------------------------
MAE: 8.063 +/- 4.338
RMSE: 13.059 +/- 7.362


In [16]:
param_grid = {
    
    'alpha': [0.05, 0.1],
    'beta' :[0.01, 0.1, 0.25],
    'lambda1' : [0.0000001, 0.001],
    'sigma' :[0.5, 1, 10, 50],
    'e_utility': [0.03, 0.05]
}

In [28]:
(train_index, test_index) = all_splits[-1]

In [29]:


X_train, X_test = X.values[train_index], X.values[test_index]
y_train, y_test = y[train_index], y[test_index]

In [30]:
from sklearn.model_selection import GridSearchCV
from ePL_KRLS_DISCO import ePL_KRLS_DISCO

model = ePL_KRLS_DISCO()
ts = GridSearchCV(model, parameters)



In [13]:
param_grid.values()

dict_values([[0.05, 0.1], [0.01, 0.1, 0.25], [1e-07, 0.001], [0.5, 1, 10, 50], [0.03, 0.05]])

In [59]:
product(*param_grid.values())

In [63]:
from itertools import product

param_names = list(param_grid.keys())
param_combinations = list(product(*param_grid.values()))

In [14]:
from sklearn.model_selection import ParameterGrid
grid = ParameterGrid(param_grid)

In [15]:
grid

In [21]:
from itertools import product

def grid_search_ePL_KRLS_DISCO(X, y, param_grid, splits):
    print("Iniciando Grid Search - ePL_KRLS_DISCO")
    print("---------------------------------------------------------------------------------\n")
    
    best_score = float('inf')
    best_params = None
    all_results = []

    # Gerar todas as combinações possíveis de hiperparâmetros
    param_names = list(param_grid.keys())
    param_combinations = list(product(*param_grid.values()))

    for i, param_values in enumerate(param_combinations):
        print(f"Testando combinação {i+1}/{len(param_combinations)}")
        print(f"Parâmetros: {dict(zip(param_names, param_values))}")
        
        # Atualizar os hiperparâmetros do modelo
        hyperparams = dict(zip(param_names, param_values))

        # Criar uma nova função de avaliação para este conjunto de hiperparâmetros
        def avaliaModelo(X, y, splits=splits):
            from ePL_KRLS_DISCO import ePL_KRLS_DISCO
            mae_scores = np.array([])
            rmse_scores = np.array([])

            for train_index, test_index in splits:
                from ePL_KRLS_DISCO import ePL_KRLS_DISCO
                model = ePL_KRLS_DISCO(**hyperparams)
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                OutputTraining, Rules = model.fit(X_train, y_train)
                OutputTest = model.predict(X_test)

                tmp_y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1))
                tmp_prev = scaler_y.inverse_transform(OutputTest.reshape(-1, 1))

                mae = mean_absolute_error(tmp_y_test, tmp_prev)
                rmse = np.sqrt(mean_squared_error(tmp_y_test, tmp_prev))

                mae_scores = np.append(mae_scores, mae)
                rmse_scores = np.append(rmse_scores, rmse)

            return mae_scores.mean(), rmse_scores.mean()

        # Avaliar o modelo com os hiperparâmetros atuais
        mae_mean, rmse_mean = avaliaModelo(X, y)

        print(f"Resultado: MAE = {mae_mean:.3f}, RMSE = {rmse_mean:.3f}\n")

        # Guardar o resultado
        all_results.append((hyperparams, mae_mean, rmse_mean))

        # Verificar se é a melhor combinação de hiperparâmetros até agora
        if rmse_mean < best_score:
            best_score = rmse_mean
            best_params = hyperparams

    print("Grid Search concluído")
    print("---------------------------------------------------------------------------------")
    print(f"Melhores parâmetros encontrados: {best_params}")
    print(f"Melhor RMSE: {best_score:.3f}\n")

    return best_params, best_score, all_results


best_params, best_score, all_results = grid_search_ePL_KRLS_DISCO(X.values, y, param_grid, all_splits)

Iniciando Grid Search - ePL_KRLS_DISCO
---------------------------------------------------------------------------------

Testando combinação 1/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 1e-07, 'sigma': 0.5, 'e_utility': 0.03}
Resultado: MAE = 4.147, RMSE = 7.654

Testando combinação 2/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 1e-07, 'sigma': 0.5, 'e_utility': 0.05}
Resultado: MAE = 4.273, RMSE = 7.249

Testando combinação 3/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 1e-07, 'sigma': 1, 'e_utility': 0.03}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7286541887635238' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.parameters.at[i, 'nu'] = math.sqrt((self.parameters.loc[i, 'nu'])**2 + (((np.linalg.norm(x - self.parameters.loc[i, 'Center']))**2 - (self.parameters.loc[i, 'nu'])**2)/self.parameters.loc[i, 'NumObservations']) + ((self.parameters.loc[i, 'NumObservations'] - 1) * ((np.linalg.norm(self.parameters.loc[i, 'Center'] - self.parameters.loc[i, 'OldCenter']))**2))/self.parameters.loc[i, 'NumObservations'])
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7286541887635238' has dtype incompatible with int64, please explicitly cast to a compatible dtype 

Resultado: MAE = 9.537, RMSE = 16.090

Testando combinação 4/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 1e-07, 'sigma': 1, 'e_utility': 0.05}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7286541887635238' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.parameters.at[i, 'nu'] = math.sqrt((self.parameters.loc[i, 'nu'])**2 + (((np.linalg.norm(x - self.parameters.loc[i, 'Center']))**2 - (self.parameters.loc[i, 'nu'])**2)/self.parameters.loc[i, 'NumObservations']) + ((self.parameters.loc[i, 'NumObservations'] - 1) * ((np.linalg.norm(self.parameters.loc[i, 'Center'] - self.parameters.loc[i, 'OldCenter']))**2))/self.parameters.loc[i, 'NumObservations'])
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7286541887635238' has dtype incompatible with int64, please explicitly cast to a compatible dtype 

Resultado: MAE = 11.104, RMSE = 18.230

Testando combinação 5/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 1e-07, 'sigma': 10, 'e_utility': 0.03}


...

Resultado: MAE = 1.780, RMSE = 2.492

Testando combinação 14/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 0.001, 'sigma': 10, 'e_utility': 0.05}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.07325504465961' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.parameters.at[i, 'nu'] = math.sqrt((self.parameters.loc[i, 'nu'])**2 + (((np.linalg.norm(x - self.parameters.loc[i, 'Center']))**2 - (self.parameters.loc[i, 'nu'])**2)/self.parameters.loc[i, 'NumObservations']) + ((self.parameters.loc[i, 'NumObservations'] - 1) * ((np.linalg.norm(self.parameters.loc[i, 'Center'] - self.parameters.loc[i, 'OldCenter']))**2))/self.parameters.loc[i, 'NumObservations'])
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.07325504465961' has dtype incompatible with int64, please explicitly cast to a compatible dtype firs

Resultado: MAE = 1.766, RMSE = 2.439

Testando combinação 15/96
Parâmetros: {'alpha': 0.05, 'beta': 0.01, 'lambda1': 0.001, 'sigma': 50, 'e_utility': 0.03}


d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '35.35577657083497' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.parameters.at[i, 'nu'] = math.sqrt((self.parameters.loc[i, 'nu'])**2 + (((np.linalg.norm(x - self.parameters.loc[i, 'Center']))**2 - (self.parameters.loc[i, 'nu'])**2)/self.parameters.loc[i, 'NumObservations']) + ((self.parameters.loc[i, 'NumObservations'] - 1) * ((np.linalg.norm(self.parameters.loc[i, 'Center'] - self.parameters.loc[i, 'OldCenter']))**2))/self.parameters.loc[i, 'NumObservations'])
d:\Dissertação\dissertacao_pgmc\Cadernos\..\Modelo\ePL_KRLS_DISCO.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '35.35577657083497' has dtype incompatible with int64, please explicitly cast to a compatible dtype fi

KeyboardInterrupt: 